In [1]:
!pip install fredapi

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\rahul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\rahul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [32]:
!nvidia-smi

Thu Nov 14 16:59:36 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   43C    P3             12W /   55W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
import pandas as pd
import torch 
import matplotlib.pyplot as plt
from fredapi import Fred
import numpy as np

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["API_KEY"]

In [6]:
fred = Fred(api_key=api_key)
data = fred.get_series("NROU")
data.tail()

2033-10-01    4.236570
2034-01-01    4.230920
2034-04-01    4.225190
2034-07-01    4.219381
2034-10-01    4.213503
dtype: float64

In [7]:
data = pd.DataFrame(data,columns=['Rate'])
data.reset_index(inplace=True)
data.head()

,index,Rate
0,1949-01-01,5.255053
1,1949-04-01,5.261516
2,1949-07-01,5.268013
3,1949-10-01,5.274564
4,1950-01-01,5.281182


In [8]:
data.rename(columns={"index": "Date"}, inplace=True)
data.head()

,Date,Rate
0,1949-01-01,5.255053
1,1949-04-01,5.261516
2,1949-07-01,5.268013
3,1949-10-01,5.274564
4,1950-01-01,5.281182


In [26]:
rates = data["Rate"].to_numpy()
rates

array([5.25505257, 5.26151562, 5.268013  , 5.274564  , 5.281182  ,
       5.287866  , 5.294604  , 5.301367  , 5.308113  , 5.314789  ,
       5.32133341, 5.3276825 , 5.333774  , 5.339555  , 5.34498549,
       5.35004044, 5.354713  , 5.35901165, 5.362958  , 5.36658239,
       5.369922  , 5.37301445, 5.375902  , 5.378624  , 5.38122   ,
       5.383725  , 5.386171  , 5.388586  , 5.39099646, 5.39343   ,
       5.395917  , 5.398493  , 5.40119839, 5.40408039, 5.4071846 ,
       5.410558  , 5.41424036, 5.418266  , 5.42265844, 5.427429  ,
       5.43257666, 5.43808556, 5.443927  , 5.450064  , 5.45645237,
       5.463051  , 5.46982956, 5.47677   , 5.483875  , 5.491165  ,
       5.49867535, 5.50645542, 5.514559  , 5.523042  , 5.53195047,
       5.541324  , 5.551188  , 5.561552  , 5.572411  , 5.58374739,
       5.59552765, 5.607706  , 5.62022257, 5.633007  , 5.645977  ,
       5.659044  , 5.672116  , 5.685105  , 5.69793   , 5.710529  ,
       5.722861  , 5.734915  , 5.746704  , 5.75827   , 5.76967

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
cut_batch = int(len(rates) * .6)
end_batch = int(len(rates) * .2)

X_train = torch.from_numpy(rates).to(device)
X_eval = data[cut_batch:(cut_batch+ end_batch)]
X_test = data[(cut_batch+ end_batch):]

len(X_train), len(X_eval), len(X_test)

(344, 68, 70)

In [10]:
import torch.nn as nn
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer = nn.Linear(in_features=1, out_features=1)

    def forward(self,x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

In [ ]:
model0 = LinearRegressionModel()

loss_fn = nn.L1Loss()

optimizer = torch.optim.SGD(model0.parameters(), lr=.01)

In [ ]:
def noLossFr():
    model0.train()
    x_pred = model0.forward(X_train)
    loss = loss_fn()
